In [1]:
import os
import pandas as pd
import numpy as np



In [2]:
path = 'Bloomberg DATI\\DATI FACTSET\\new\\'

files = os.listdir(path)


#### READ OUTPUT ETF RAW DATA FROM EXCEL FILE AND ADJUST COLUMNS AND ROWS

def read_data(file, skiprows=15):
    name = path + file
    data = pd.read_excel(name, skiprows=skiprows)
    data = data[['Date', 'Price', 'Open', 'High', 'Low', 'CVol']]
    data = data.dropna()
    data = data[1:]
    cols = ['Date', 'Open', 'High', 'Low', 'Price', 'CVol']
    data = data[cols]
    data.columns = ['Date', 'Open', 'High', 'Low', 'Adj Close', 'Vol']
    
    return data

##### Set Up egual starting point date (max_start_date)
start_dates = []
for file in files:
    data = read_data(file)
    start_date = data.Date.tail(1).tolist()[0]
    start_dates.append(start_date)
        
max_start_date = max(start_dates)

all_data = []

##### Select Data from starting point
for file in files:
    ticker = file.split('.')[0]
    data = read_data(file)
    filter_data = data.Date>=max_start_date
    data = data[filter_data]
    data['Ticker'] = ticker
    # create a list of data tables 
    all_data.append(data)
    
    # save adjusted data as .csv file
    #data.to_csv(ticker)
    
output_ETF = all_data


In [6]:
#### READ INPUT RAW DATA FROM EXCEL FILE AND ADJUST COLUMNS AND ROWS

path = 'input\\Price Indexes\\'
files = os.listdir(path)
files

def read_data(file, skiprows=3):
    name = path + file
    data = pd.read_excel(name, skiprows=skiprows)
    data = data.iloc[:,:2]
    data = data.dropna()
    data.columns = ['Date', 'Close']
   
    return data

def read_data_fund(file, skiprows=3):
    name = 'input\\Fundamentals - Yearly\\' + file
    data = pd.read_excel(name, skiprows=skiprows)
    data = data.iloc[:,:2]
    data = data.dropna()
    indicator_name = file.split(".")[0]
    data.columns = ['Date', indicator_name]
    data['Date'] = pd.to_datetime(data['Date'], format='%m/%d/%Y')
    
    return data

list_indicator = ['US_10-2_Year_Yield', 'US_2_Year-3_Month_Yield', 'Copper/Gold', 'Sp_500/Dow_Jones', 'Sp_500/Russel_2000', 'Jpy', 'US_10_Year_Yield', 'Gold', 'Copper', 'Oil', 'Sp_500', 'Fund:CPI', 'Fund:GDP Growth', 'Fund:ISM', 'Fund:CCI', 'Fund:Unemployment', 'Fund:Us Non Farm Payroll']
all_data = []
for indicator in list_indicator:
    if len(indicator.split('/'))>1:
        file1 = indicator.split('/')[0] + '.xlsx'
        data1 =  read_data(file1)
        data1.set_index('Date', inplace=True)
        file2 = indicator.split('/')[1] + '.xlsx'
        data2 =  read_data(file2)
        data2.set_index('Date', inplace=True)
        data = data1/data2
        data = data.dropna()
        data = data.sort_values('Date', ascending=False)
        data.columns = [indicator]
    elif len(indicator.split(':'))>1:
        file = indicator.split(':')[1] + '.xlsx'
        data = read_data_fund(file)
        data.set_index('Date', inplace=True)
    else:
        file = indicator + '.xlsx'
        data = read_data(file)
        data.columns = ['Date', indicator]
        data.set_index('Date', inplace=True)
        
    ######### Select Input Data from starting point #########
    #filter_data = data.Date>=max_start_date
    #data = data[filter_data]
    #########################################################   


    # create a list of data tables
    if len(all_data)>0:
        try:
            all_data[indicator] = data[indicator]
        except:
            indicator_name = indicator.split(':')[1]
            all_data[indicator_name] = data[indicator_name]
            if indicator_name == 'Us Non Farm Payroll':
                all_data[indicator_name] = all_data[indicator_name].fillna(method='ffill')
            #all_data[indicator_name] = all_data[indicator_name].fillna(method='ffill')
    else:
        all_data = data


input_data = all_data.dropna()

In [7]:
output_ETF[0]


,Date,Open,High,Low,Adj Close,Vol,Ticker
1,2019-07-22,83.020,83.0300,82.9200,82.950,1725513.0,BND
2,2019-07-19,82.850,82.9200,82.8123,82.860,2167269.0,BND
3,2019-07-18,82.770,82.9800,82.7300,82.950,3323190.0,BND
4,2019-07-17,82.670,82.8100,82.6400,82.800,2062285.0,BND
5,2019-07-16,82.520,82.5800,82.4550,82.550,2294091.0,BND
6,2019-07-15,82.530,82.6600,82.5300,82.660,2245920.0,BND
7,2019-07-12,82.470,82.5700,82.4434,82.540,2265037.0,BND
8,2019-07-11,82.710,82.7700,82.4300,82.470,3024150.0,BND
9,2019-07-10,82.770,82.8100,82.6800,82.740,2097173.0,BND
10,2019-07-09,82.790,82.7900,82.6600,82.670,1839373.0,BND


In [8]:
input_data


,US_10-2_Year_Yield,US_2_Year-3_Month_Yield,Copper/Gold,Sp_500/Dow_Jones,Sp_500/Russel_2000,Jpy,US_10_Year_Yield,Gold,Copper,Oil,Sp_500,CPI,GDP Growth,ISM,CCI,Unemployment,Us Non Farm Payroll
Date,,,,,,,,,,,,,,,,,
2018-12-31,0.1836,0.0516,0.002057,0.107463,0.747976,109.69,2.6836,1284.7000,2.6430,45.81000,2506.847183,2.4354,2.857186,58.750000,126.6,3.891667,150275.0
2017-12-29,0.5495,0.4969,0.002523,0.108159,0.700607,112.69,2.4325,1305.0000,3.2930,60.40000,2673.610523,2.13594,2.216975,57.441666,123.1,4.350000,147767.0
2016-12-30,1.2730,0.7429,0.002178,0.113286,0.663787,116.96,2.4752,1152.0000,2.5085,53.89000,2238.826682,1.27002,1.567290,51.358334,113.3,4.875000,145695.0
2015-12-31,1.2404,0.8701,0.002013,0.117299,0.724038,120.22,2.2961,1060.5000,2.1350,37.07000,2043.936863,0.119483,2.880910,51.283333,96.3,5.275000,143125.0
2014-12-31,1.5197,0.6456,0.002388,0.115519,0.687682,119.78,2.1881,1183.2000,2.8260,53.71000,2058.902379,1.61546,2.451973,55.458332,93.0629,6.158333,140396.0
2013-12-31,2.5894,0.3257,0.002821,0.111504,0.639142,105.31,2.9734,1204.8000,3.3990,98.70000,1848.356521,1.46596,1.842081,53.758335,77.5355,7.358333,137390.0
2012-12-31,1.5050,0.2423,0.002178,0.108835,0.675647,86.75,1.7600,1675.6000,3.6500,91.79000,1426.187978,2.07319,2.249546,51.683334,66.6932,8.075000,135088.0
2011-12-30,1.6361,0.2277,0.002189,0.102934,0.682975,76.91,1.8791,1567.0000,3.4300,99.06000,1257.604805,3.13965,1.550835,55.183334,64.7574,8.933333,133269.0
2010-12-31,2.6967,0.4666,0.003129,0.108628,0.645749,81.12,3.2901,1421.4000,4.4470,91.38000,1257.635988,1.63658,2.563766,57.283333,63.4016,9.608334,130839.0
